In [ ]:
!pip install music21 pretty_midi keras-self-attention

In [2]:
from music21 import *
import numpy, os, pretty_midi, glob, pathlib, pickle, random, multiprocessing
from multiprocessing import Pool
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, Activation, BatchNormalization as BatchNorm
from keras_self_attention import SeqSelfAttention
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split


"""
    Prepare the sequences used by the Neural Network.

    Args:
        music_notes (list): List of notes or chords.
        total_unique_notes (int): Total number of unique pitches or chords.

    Returns:
        tuple: A tuple containing the network input sequences and corresponding network output.

"""
def prepare_Note_Sequences(music_notes, total_unique_notes):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all unique note names and sort them
    unique_notes = sorted(set(music_notes))

    # Create a dictionary to map notes to integers
    note_to_int_mapping = {note: number for number, note in enumerate(unique_notes)}

    input_sequences = []
    output_sequences = []

    # Create input sequences and the corresponding outputs
    for i in range(len(music_notes) - sequence_length):
        input_sequence = music_notes[i:i + sequence_length]
        output_note = music_notes[i + sequence_length]
        input_sequences.append([note_to_int_mapping[note] for note in input_sequence])
        output_sequences.append(note_to_int_mapping[output_note])

    # Reshape the input into a format compatible with LSTM layers and normalize it
    normalized_input = numpy.reshape(input_sequences, (len(input_sequences), sequence_length, 1)) / float(total_unique_notes)

    # Convert the output to categorical data
    categorical_output = np_utils.to_categorical(output_sequences)

    return (normalized_input, categorical_output)

"""
    Create the structure of the neural network for generating hand movements.

    Args:
        network_input (ndarray): Input sequences for the network.
        n_vocab (int): Total number of unique pitches or chords.
        name (str): Name used for loading pre-trained weights.

    Returns:
        keras.models.Sequential: The compiled model for generating hand movements.

"""
def create_Hand_Model(network_input, n_vocab,name):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))  # Add attention layer
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    model.load_weights(f'/content/drive/MyDrive/weights-{name}.hdf5')

    return model

def main():
     # Set the directory paths for parts, left hand, and right hand
    part_dir = '/content/drive/MyDrive/Parts'
    left_dir = 'Left'
    right_dir = 'Right'

    # Load the right hand notes from the pickle file
    with open('/content/drive/MyDrive/Parts/notes-Right.pkl', 'rb') as filepath:
        right_notes = pickle.load(filepath)
    
    # Load the left hand notes from the pickle file
    with open('/content/drive/MyDrive/Parts/notes-Left.pkl', 'rb') as filepath:
        left_notes = pickle.load(filepath)
    
    # Calculate the number of unique pitches or chords for the right and left hand
    right_vocab = len(set(right_notes))
    left_vocab = len(set(left_notes))

    # Prepare the sequences for the right hand model
    right_hand_input, right_hand_output = prepare_Note_Sequences(right_notes, right_vocab)
    right_train_input, right_input_val, right_output_train, right_output_val = train_test_split(right_hand_input, right_hand_output, test_size=0.2, random_state=42)

    # Prepare the sequences for the left hand model
    left_hand_input, left_hand_output = prepare_Note_Sequences(left_notes, left_vocab)
    left_train_input, left_input_val, left_output_train, left_output_val = train_test_split(left_hand_input, left_hand_output, test_size=0.2, random_state=42)

    # Create the right hand model
    right_model = create_Hand_Model(right_hand_input, right_vocab, 'Right')

    # Create the left hand model
    left_model = create_Hand_Model(left_hand_input, left_vocab, 'Left')

    # Train the right hand model
    train(right_model, right_hand_input, right_hand_output, "Right",(right_input_val, right_output_val))

    # Train the left hand model
    train(left_model, left_hand_input, left_hand_output, "Left",(left_input_val, left_output_val))


"""
    Train the neural network.

    Args:
        model (keras.models.Sequential): The model to train.
        network_input (ndarray): Input sequences for training.
        network_output (ndarray): Corresponding network output for training.
        name (str): Name used for saving the trained weights.
        validation_data (ndarray): validation data

    Returns:
        None

"""
def train(model, network_input, network_output, name, validation_data):
    filepath = f"/content/drive/MyDrive/weights-{name}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1)
    callbacks_list = [checkpoint, early_stop]

    model.fit(network_input, network_output, epochs=20, batch_size=1408, shuffle=True,
              validation_data=validation_data, callbacks=callbacks_list)

if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Epoch 1/20
105/105 [==============================] - 77s 596ms/step - loss: 0.8808 - val_loss: 0.1795
Epoch 2/20
105/105 [==============================] - 59s 563ms/step - loss: 0.8055 - val_loss: 0.1637
Epoch 3/20
105/105 [==============================] - 59s 565ms/step - loss: 0.8003 - val_loss: 0.1413
Epoch 4/20
105/105 [==============================] - 59s 567ms/step - loss: 0.7816 - val_loss: 0.2001
Epoch 5/20
105/105 [==============================] - 59s 564ms/step - loss: 0.7801 - val_loss: 0.1840
Epoch 6/20
105/105 [==============================] - 60s 567ms/step - loss: 0.7664 - val_loss: 0.1420
Epoch 7/20
105/105 [==============================] - 58s 555ms/step - loss: 0.7734 - val_loss: 0.1875
Epoch 8/20
105/105 [==============================] - 58s 555ms/step - loss: 0.7703 - val_loss: 0.1564
Epoch 9/20
105/105 [==============================] - 58s 555ms/step - loss: 0.7719 - val_loss: 0.1471
Epoch 10/20
105/105 [==============================] - 58s 554ms/step - l